In [1]:
# Create a folder called 'Fintech_Images' and add your csv and ipynb files under the same directory
import pandas as pd
import requests
from PIL import Image
import os

import ast
import io
from google.colab import drive

# Multiprocesser for faster images processing 
from multiprocessing import Pool # TODO: See if we can do multiprocessing

In [2]:
drive.mount('/content/drive') # Connet to your google drive

folder_name = "Fintelics_Image" # Your current folder name 

img_folder_name = "Fintelic_Image"

Mounted at /content/drive


In [5]:
import glob
print(glob.glob("/content/drive/MyDrive/" + folder_name + "/*")) # You should see 2 file paths
path = glob.glob("/content/drive/MyDrive/" + folder_name + "/*")[1] # csv path

['/content/drive/MyDrive/Fintelics_Image/cnn_model.ipynb', '/content/drive/MyDrive/Fintelics_Image/103711_nfts.csv', '/content/drive/MyDrive/Fintelics_Image/Fintelic_Image', '/content/drive/MyDrive/Fintelics_Image/Fintelics_Image_60001_80000', '/content/drive/MyDrive/Fintelics_Image/Fintelics_features']


In [6]:
nfts = pd.read_csv(path)
nfts.head()

,tokenContractAddress,tokenName,tokenSymbol,tokenId,tokenMetadatURI,tokenOWner,tokenMetadataJSON
0,0xF216c9E7414E84a37F5c24148fa8899CB674AB84,Unofficial Punks V2,PUNX,1697,https://nft.bueno.art/api/contract/QL56SC9m9OV...,0xdA673be6917f6264F37093F96cd1cFca7A973706,"{""attributes"":[{""value"":""Gold"",""trait_type"":""E..."
1,0xbB5dA6DE3d235a45909D8e91b86069D114C72D5f,Rekts - RG Edition,RKTS,190,https://arweave.net/iNlUsMf2KZGYEU-dlakGnU7HAP...,0xe733EF9EDf8E78E53416963e52C0c52506Ae3f72,"{""name"":""Rekts - RG Edition"",""created_by"":""pep..."
2,0xbB5dA6DE3d235a45909D8e91b86069D114C72D5f,Rekts - RG Edition,RKTS,192,https://arweave.net/iNlUsMf2KZGYEU-dlakGnU7HAP...,0xe733EF9EDf8E78E53416963e52C0c52506Ae3f72,"{""name"":""Rekts - RG Edition"",""created_by"":""pep..."
3,0xbB5dA6DE3d235a45909D8e91b86069D114C72D5f,Rekts - RG Edition,RKTS,188,https://arweave.net/iNlUsMf2KZGYEU-dlakGnU7HAP...,0xe733EF9EDf8E78E53416963e52C0c52506Ae3f72,"{""name"":""Rekts - RG Edition"",""created_by"":""pep..."
4,0xbB5dA6DE3d235a45909D8e91b86069D114C72D5f,Rekts - RG Edition,RKTS,189,https://arweave.net/iNlUsMf2KZGYEU-dlakGnU7HAP...,0xe733EF9EDf8E78E53416963e52C0c52506Ae3f72,"{""name"":""Rekts - RG Edition"",""created_by"":""pep..."


In [7]:
nfts['tokenSymbol'].value_counts()

SEWER       10323
TPLRMP       7332
CC0FREE      2575
CHCKSOUT     1984
AAS          1823
            ...  
CHIP            1
GUIX            1
T2GNFT          1
WB              1
JPEGS           1
Name: tokenSymbol, Length: 4242, dtype: int64

In [ ]:
def resize_img(file_in, file_out):
  width = 32
  height = 32
  image = Image.open(file_in)
  re_img = image.resize((width,height),Image.ANTIALIAS)
  re_img.save(file_out)

In [ ]:
from requests.api import request
# Target: convert ipfs://QmQMg8MvFUJCnLojiFx1cGy9qgDStrEe3fg222p1Z8WmpL/3661.png
# To: https://ipfs.io/ipfs/QmQMg8MvFUJCnLojiFx1cGy9qgDStrEe3fg222p1Z8WmpL/3661.png
def convert_ipfs_to_https(prev_url: str):
  if not prev_url.startswith("ipfs"):
    return prev_url
  # Get hash code 
  hash_code = prev_url.split("ipfs://")[1]
  return "https://ipfs.io/ipfs/" + hash_code #Not sure https is always correct
print(convert_ipfs_to_https("ipfs://QmQMg8MvFUJCnLojiFx1cGy9qgDStrEe3fg222p1Z8WmpL/3661.png"))

https://ipfs.io/ipfs/QmQMg8MvFUJCnLojiFx1cGy9qgDStrEe3fg222p1Z8WmpL/3661.png


In [ ]:
def safe_literal_eval(node):
     try:
         return ast.literal_eval(node)
     except ValueError:
         return None  # happens when literal eval cannot process node

In [ ]:
def image_write(start, end, nfts):
  # pick image URL out
  url_csv = nfts
  for i in range(start, end):
    url_csv = nfts['tokenMetadatURI'][i]
    try:
      url_csv = convert_ipfs_to_https(url_csv)
      print(url_csv)
      x = requests.get(url_csv)
    except Exception:
      print(f"({i}): Can not process the url_csv {url_csv}\n")
      continue
    text = x.text
    data = safe_literal_eval(text)
    if  data == None:
      continue
    if 'image' not in data.keys():
      print("No image tag!\n")
      return
    img_url = data['image']
    index1 = text.index('"image":')
    index2 = text[index1+9:].index('"')
    # img_url = text[index1+9:index1+9+index2] #extract image url
    try:
      img_url = convert_ipfs_to_https(img_url) # Processing ipfs url
      response = requests.get(img_url)
    except Exception:
      print(f"({i}): Can not process the url_img {img_url}\n")
      continue

    img = response.content
    path = "/content/drive/MyDrive/"+ folder_name +'/'+ img_folder_name +"/%i.png" % i 
    print(path)
    with open(path,'wb') as f:
      f.write(img)
    try:
      resize_img(path,path)
    except Exception:
      continue

In [ ]:
import base64

In [ ]:
nfts['data'] = nfts['tokenMetadatURI'].str.contains('data:')
nfts.loc[nfts['data'] == True, 'data'] = 'data'
nfts.loc[nfts['data'] == False, 'data'] = None

In [ ]:
nfts['data'].value_counts()
data = nfts

In [ ]:
data = data.dropna(axis=0, how = 'any')

In [ ]:
data['tokenMetadatURI'][13].index('"image":')

319

In [ ]:
web = data['tokenMetadatURI'][13]

In [ ]:
web

'data:application/json;utf8,{"name":"Receipt 0.0129 ETH","description":"Receipt is a conceptual art using NFT. The amount you pay determines the visual. You can pay any amount. one token can be minted per price. The code to draw is executed completely on-chain. The transaction itself is your participation in the art.","image":"data:image/svg+xml;utf8,<svg viewBox=\'0 0 300 460\' width=\'300\' height=\'460\' fill=\'#1a1a1a\' preserveAspectRatio=\'xMidYMid meet\' version=\'2\' xmlns=\'http://www.w3.org/2000/svg\'><style>@font-face {font-family:\'RCPT\';font-display:block;src:url(data:application/font-woff2;charset=utf-8;base64,d09GMgABAAAAABP4AAoAAAAAJtgAABOqAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAABmAANAq8YK5qATYCJAODBAuDCAAEIAWHEAcgG6sdIxH1i9TCk/0lpnDtQBpnJxwtsc1gB3ajs6txeBgeLgIAQHG93iN+AADYpSgAsMA2wIJpAYChlP73+7HOu/9+RURkOlSSqEbNeKTRGJLYbqNtMhmetvlvQ1SMRhCVqFBA4Lgg76hWJ+XQXmT+yvZXF7Hwrh4AhprN/3f6ep/trHe/T1AHLaGFpWU5kuWROQj5ohbNGoNm446DdYZcV/LDmnU4W8sZu5O6B0B3d0+2ACDRiCr3gK4EuJA0ramn/36Zs/tCK8U

In [ ]:
web[328:15464]

"data:image/svg+xml;utf8,<svg viewBox='0 0 300 460' width='300' height='460' fill='#1a1a1a' preserveAspectRatio='xMidYMid meet' version='2' xmlns='http://www.w3.org/2000/svg'><style>@font-face {font-family:'RCPT';font-display:block;src:url(data:application/font-woff2;charset=utf-8;base64,d09GMgABAAAAABP4AAoAAAAAJtgAABOqAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAABmAANAq8YK5qATYCJAODBAuDCAAEIAWHEAcgG6sdIxH1i9TCk/0lpnDtQBpnJxwtsc1gB3ajs6txeBgeLgIAQHG93iN+AADYpSgAsMA2wIJpAYChlP73+7HOu/9+RURkOlSSqEbNeKTRGJLYbqNtMhmetvlvQ1SMRhCVqFBA4Lgg76hWJ+XQXmT+yvZXF7Hwrh4AhprN/3f6ep/trHe/T1AHLaGFpWU5kuWROQj5ohbNGoNm446DdYZcV/LDmnU4W8sZu5O6B0B3d0+2ACDRiCr3gK4EuJA0ramn/36Zs/tCK8UFo5By/9u9zfz7vKslrdQlKucuKHClqqr2aKWpwyWWYTzC0jXCVK0CQgK5La8efedUIshH3zXeFHyXQk606VW2+kS/uApwNCYWIQNUUyMASvbvW/RCGxcAUdZZZiIGMKQ6acHD0WWZYN/5HPbvmbkcIQAQa+8M9J2WgCf4BSQSCzAFIxCAsq8vQOi//f//ORAZ/z//s4NWia4ruGk6A/pvANNVKlnWGARo55cgpx0gKIFAREji1e5UOfWqgy+V0YmiDbs3UV+bpNOSqVLCpi7rygKNwqZV2RRuM7cwNX3ArY4Gintkc4u3d0cD63CfR7IuRmVWvUwd5hZsbVA8TZpMDW8

In [ ]:
encode_image = web[328:15464]
decode_image = base64.b64decode(encode_image)
path = "/content/drive/MyDrive/"+ folder_name +'/'+ img_folder_name +"/%13.png" % 13
with open(path,'wb') as f:
  f.write(decode_image)

ValueError: ignored

In [ ]:
for i in range(60000,60001):
  try:
    image_write(i, i+1, nfts)
  except Exception:
    continue

https://api.mdvmm.xyz/sewerpasses/1267
/content/drive/MyDrive/Fintelics_Image/Fintelic_Image/60000.png


In [ ]:
nfts['tokenMetadatURI'][41946]

'ipfs://QmZsnxhwwcuSMdHccg2Dpkmw41Q1tr23MHQMXzC9UGZ8kQ/405.json'